In [ ]:
# Based on june-23-final.ipynb
# Handles async processing of Gemini models

import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["ANTHROPIC_API_KEY"] = ""
os.environ["GOOGLE_API_KEY"] = ""

In [2]:
#!pip install anthropic dotenv
#!pip install --upgrade datasets fsspec huggingface_hub

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


In [3]:
# REFACTOR: Standard imports + asyncio and nest_asyncio for Jupyter compatibility.
import pandas as pd
import numpy as np
import time
import math
import datetime
import importlib
import inspect
import os
import re
import json
import random
import openai
from google import genai
import anthropic
import asyncio
import nest_asyncio
from openai import AsyncOpenAI # REFACTOR: Import async client
from anthropic import AsyncClient # REFACTOR: Import async client
from datasets import load_dataset
from tqdm.notebook import tqdm # REFACTOR: Use notebook-friendly tqdm
from tqdm.asyncio import tqdm as tqdm_asyncio # Alias to avoid name conflict with tqdm.notebook if used elsewhere
from dotenv import load_dotenv
from typing import List, Dict, Any, Optional
from pathlib import Path
import textwrap, hashlib

In [ ]:
def find_project_root():
    """Traverse upwards to find the project root, marked by the .git folder."""
    current_path = Path.cwd()
    while current_path != current_path.parent:
        if (current_path / ".git").is_dir():
            return current_path
        current_path = current_path.parent
    raise FileNotFoundError("Could not find project root. Is this a git repository?")

# REFACTOR: Apply nest_asyncio to allow asyncio to run in a Jupyter notebook.
# This must be done once per kernel.
nest_asyncio.apply()

# --- 1. Client and Model Configuration ---

load_dotenv()

# REFACTOR: Initialize asynchronous clients.
# The synchronous clients are no longer needed for the generation script.
openai_client_async = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))
anthropic_client_async = AsyncClient(api_key=os.getenv("ANTHROPIC_API_KEY"))

# Initialize the global Gemini client (used for cache operations)
# It will pick up the API key configured via genai.configure()
gemini_client: Optional[genai.Client] = None
# This will store a dictionary of cached_prompt_ids, keyed by model name
gemini_model_caches: Dict[str, Optional[str]] = {}
# The models used for caching will now be taken directly from the model_dict

# Define the project root as a global constant
PROJECT_ROOT = find_project_root()
print(f"Project root identified: {PROJECT_ROOT}")
BASE_OUTPUT_DIR = PROJECT_ROOT / 'data' / 'code_gen_outputs_raw'

# A safe starting point is slightly below the documented RPM.
API_CONCURRENCY_LIMITS = {
    "google":    2,   # 2 parallel calls ≈ 30 RPM  < 10 RPM × 3 models
    "anthropic": 2,   # 2 × 3 500 tokens × 60/4 ≈ 105 k TPM → still too high
    "openai":    2,   # unchanged
}


Project root identified: content


In [5]:
# --- 2. System Prompt and Helper Functions (no changes needed) ---

SYSTEM_PROMPT = "You are an expert Python programmer specializing in data formalization. Your role is to meticulously convert natural language math problems and their step-by-step solutions into a single, well-structured Python function. You will be presented with examples of the required format followed by a final task to complete."


PROMPT_GUIDELINES = """### Guidelines

0. **Output wrapping**
   Return the code inside a single ```python … ``` block, and nothing else.

1.  **Function Naming & Docstring:** The function must be named `solve`. It must begin with a docstring that has exactly two lines:
    *   The first line must be: "Index: [Index]." using the index from the task header.
    *   The second line must be a succinct, one-sentence description of what the function returns (e.g., "Returns: the total cost of wages and taxes.").

2.  **Function Arguments:** The function arguments must be derived from the 'Question' text.
    *   Create a distinct argument for every numerical value that is directly stated in the text.
    *   The arguments should be created **in the same order in which they appear in the question**.
    *   **Note:** Some of these arguments may end up not being used in the function body. This is expected. Do not worry about this and leave the unused arguments in the function signature.

3.  **Argument Formatting:** Each argument must include a type-hint (e.g., `int`, `float`) and a default value equal to its value in the 'Question'. You must also add a comment (`#`) next to each argument that quotes or refers to the phrase in the 'Question' it comes from.

4.  **Function Body:** The body of the function should follow the logic of the provided 'Solution' dict, which contains the step-by-step solution to the problem. The keys of this dict are strings (e.g. `"L1"`, `"L2"`) which refer to the line number, and the values of the dict are the corresponding steps in the solution.
    * For every relevant line in the 'Solution', you must include a comment in the Python code that indicates the line number (key) from the 'Solution' dict.
    * These comments should be formatted as `#: L<n>`, where `<n>` is the line number from the 'Solution' dict.
    * Immediately follow the comment with the Python statement that performs the calculation.
    * Steps in the solution should result in the creation of new, intermediate variables, which should be named descriptively based on the context of the calculation.
    * Wherever possible, in your code try to use only the variables from the function arguments and the intermediate variables you created before, and try to avoid using hard-coded numbers in the calculations.

5.  **Calculator Annotations:** Pay close attention to the calculator annotations (e.g., `[[25*8=200]]`) in the 'Solution' as they reveal the precise mathematical operations to implement. **Note**: Some lines in the solution may not contain calculator annotations, but you should still pay attention to the logic and calculations described in those lines.

6.  **Final Answer:** Store the final answer in a variable named 'answer', and on the same line, add the comment `# FINAL ANSWER`. In the next line, return the 'answer' variable.

7. **No extra output:** Your output should end with the ``` closing the code block. Do not include any additional text, explanations, or comments outside of the code block."""

gsm8k_train = load_dataset("gsm8k", "main", split="train")

def build_solution_mapping(
    index: int,
    dataset: Any,
    convert_brackets: bool = True,
):
    """
    Parameters
    ----------
    index : int
        Position of the sample in the loaded dataset.
    dataset : iterable / HuggingFace Dataset
    convert_brackets : bool, default ``True``
        If ``True`` replace every ``<< … >>`` calculator annotation with
        the canonical ``[[ … ]]`` form so downstream code sees a single
        bracket style.

    Returns
    -------
    Dict[str, str]
        Mapping ``{"L1": <first non-empty line>, "L2": <second>, …}``.

    Notes
    -----
    * Blank lines in ``sample["answer"]`` are ignored.
    * The line numbering reflects the *order* in the original solution
      string; there is no semantic grouping beyond that.
    """
    # extract & split solution text
    solution_text = dataset[index]["answer"]
    lines = [ln.strip() for ln in solution_text.splitlines() if ln.strip()]

    # Remove the last line if it matches the '####' answer pattern
    if lines and re.match(r"^####\s*\d+(\.\d+)?$", lines[-1]):
        lines = lines[:-1]

    # optional bracket normalisation
    if convert_brackets:
        angle = re.compile(r"<<([^>]+)>>")
        lines = [angle.sub(r"[[\1]]", ln) for ln in lines]
    # build mapping
    return {f"L{i}": line for i, line in enumerate(lines, 1)}


def format_prompt_query(index: int, code_strings: dict, with_code: bool = False):
    sample = gsm8k_train[index]
    question = sample["question"]
    solution_mapping = build_solution_mapping(index, gsm8k_train)
    solution = json.dumps(solution_mapping)
    out = f"""*Index*:
{index}

*Question*:
{question}

*Solution*:
{solution}

*Code*:"""
    if with_code:
        out += f"""\n```python
{code_strings.get(index, "# Code not found")}
```"""
    return out


def get_code_strings(indices: List[int], savepath: Path = PROJECT_ROOT / 'data' / 'code_examples'):
    """
    Reads code examples directly from .py files instead of importing them.
    This is more robust and avoids Python's complex import path mechanics.
    """
    code_strings = {}
    for idx in indices:
        # Construct the full file path
        filepath = os.path.join(savepath, f"_{idx}.py")
        try:
            # Read the entire content of the file
            with open(filepath, 'r', encoding='utf-8') as f:
                code_strings[idx] = f.read()
        except FileNotFoundError:
            print(f"Error: Could not find example file for index {idx} at: {filepath}")
            code_strings[idx] = f"# Error: Code for example {idx} not found."
    return code_strings


EXAMPLE_INDICES = [310, 3822, 7371]          # keep fixed order
_CODE_EXAMPLES  = get_code_strings(EXAMPLE_INDICES)


def build_static_prefix() -> str:
    """Guidelines + few-shot examples rendered once for caching."""
    examples_block = "\n".join(
        format_prompt_query(idx, _CODE_EXAMPLES, with_code=True)
        for idx in EXAMPLE_INDICES
    )
    prefix = "\n".join([
        PROMPT_GUIDELINES.strip(),
        "\n--- EXAMPLES ---\n",
        examples_block.strip()
    ])
    # Canonical whitespace – important for cache hits
    return textwrap.dedent(prefix).strip()


STATIC_PREFIX = build_static_prefix()
print(STATIC_PREFIX)
print("Static prefix SHA-1:", hashlib.sha1(STATIC_PREFIX.encode()).hexdigest())


def build_task_prompt(index: int) -> str:
    """Problem-specific part that changes every call."""
    return "\n".join([
        "--- TASK ---\n",
        format_prompt_query(index=index,
                            code_strings={},     # no code in the task part
                            with_code=False)
    ])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### Guidelines

0. **Output wrapping**
   Return the code inside a single ```python … ``` block, and nothing else.

1.  **Function Naming & Docstring:** The function must be named `solve`. It must begin with a docstring that has exactly two lines:
    *   The first line must be: "Index: [Index]." using the index from the task header.
    *   The second line must be a succinct, one-sentence description of what the function returns (e.g., "Returns: the total cost of wages and taxes.").

2.  **Function Arguments:** The function arguments must be derived from the 'Question' text.
    *   Create a distinct argument for every numerical value that is directly stated in the text.
    *   The arguments should be created **in the same order in which they appear in the question**.
    *   **Note:** Some of these arguments may end up not being used in the function body. This is expected. Do not worry about this and leave the unused arguments in the function signature.

3.  **Argument Formatting:**

In [6]:
# --- 3. Asynchronous API Calling Function ---

# ── Anthropic token-bucket limiter ────────────────────────────────────

# one global bucket: 50 000 input-tokens per rolling minute
_anthropic_bucket = {"tokens": 50_000, "reset_at": time.monotonic() + 60}

async def _anthropic_throttle(tokens_needed: int):
    """
    Wait until at least `tokens_needed` tokens are available in the
    50 000-tokens-per-minute bucket, then deduct them.
    """
    global _anthropic_bucket

    while True:
        now = time.monotonic()

        # reset bucket if a minute passed
        if now >= _anthropic_bucket["reset_at"]:
            _anthropic_bucket = {"tokens": 50_000, "reset_at": now + 60}

        if tokens_needed <= _anthropic_bucket["tokens"]:
            _anthropic_bucket["tokens"] -= tokens_needed
            return                               # we’re clear to send
        else:
            # need to wait for bucket refill
            to_sleep = _anthropic_bucket["reset_at"] - now
            await asyncio.sleep(max(to_sleep, 0.01))


async def _with_429_retries(send_coroutine_factory, *, max_attempts=3, default_wait=15):
    """
    send_coroutine_factory : lambda  ->  coroutine
        Pass a **lambda** that, when called, returns the coroutine you want
        to execute (e.g. the gemini.generate_content_async call).

    Retries up to `max_attempts` times on 429 errors, using the
    `retry_delay` from the API error when available, otherwise `default_wait`.
    """
    attempt = 0
    wait = default_wait
    while True:
        try:
            return await send_coroutine_factory()
        except Exception as e:
            # crude but effective 429 detection
            if ("429" in str(e)) and attempt < max_attempts:
                # Google errors include retry_delay.seconds
                retry_attr = getattr(e, "retry_delay", None)
                wait = retry_attr.seconds if retry_attr else wait
                attempt += 1
                print(f"🕒 Google 429 — retrying in {wait}s (attempt {attempt}/{max_attempts})")
                await asyncio.sleep(wait)
            else:
                raise  # re-raise non-429 or maxed-out errors


async def call_model_api_async(
        provider: str,
        model: str,
        static_prefix: str,
        all_gemini_model_caches: Dict[str, Optional[str]],
        task_prompt: str,
        semaphore: asyncio.Semaphore):

    async with semaphore:
        usage = {"input_tokens": 0,
                 "output_tokens": 0,
                 "cached_tokens": 0}

        # ─────────────────── Google (no prompt cache) ───────────────────
        if provider == "google":
            # Retrieve the correct cached_prompt_id for this specific model
            gemini_cached_prompt_id = all_gemini_model_caches.get(model)

            if gemini_cached_prompt_id is None:
                raise ValueError(f"Gemini cached_prompt_id not found or failed to create for model '{model}'. Cannot proceed with explicit caching.")

            # Removed: gemini_model = GenerativeModel(model_name=model)
            # Removed: cfg = genai.types.GenerationConfig(temperature=0.1, max_output_tokens=4000)

            def gemini_call_factory():
                # FIX: Call generate_content directly on gemini_client.aio.models
                # and pass model, contents, and config explicitly.
                return gemini_client.aio.models.generate_content(
                    model=model, # Pass model name directly as an argument
                    contents=[
                        genai.types.Content(role="user", parts=[genai.types.Part(text=task_prompt)])
                    ],
                    # Use 'config' for all optional parameters, including cached_content
                    config=genai.types.GenerateContentConfig(
                        temperature=0.1, # Move temperature here
                        max_output_tokens=4000, # Move max_output_tokens here
                        cached_content=gemini_cached_prompt_id
                    )
                )

            response = await _with_429_retries(gemini_call_factory)

            text = response.text
            if response.usage_metadata:
                usage["input_tokens"]  = response.usage_metadata.prompt_token_count
                usage["output_tokens"] = response.usage_metadata.candidates_token_count
                usage["cached_tokens"] = response.usage_metadata.cached_content_token_count


        # ─────────────────── Anthropic (cache_control) ──────────────────
        elif provider == "anthropic":
            system_block = {
                "type": "text",
                "text": static_prefix,
                "cache_control": {"type": "ephemeral"},
            }

            # crude token estimate: words×1.2  (safe over-count)
            est_tokens = math.ceil(1.2 * len(static_prefix.split()))
            await _anthropic_throttle(est_tokens)

            response = await anthropic_client_async.messages.create(
                model=model,
                max_tokens=4000,
                temperature=0.1,
                system=[system_block],
                messages=[{"role": "user", "content": task_prompt}],
            )
            text = response.content[0].text
            usage["input_tokens"]   = response.usage.input_tokens
            usage["output_tokens"]  = response.usage.output_tokens
            usage["cached_tokens"]  = response.usage.cache_read_input_tokens

        # ─────────────────── OpenAI (auto prompt cache) ─────────────────
        elif provider == "openai":
            msgs = [
                {"role": "system", "content": static_prefix},
                {"role": "user",   "content": task_prompt},
            ]
            rsp = await openai_client_async.chat.completions.create(
                model=model,
                messages=msgs,
                temperature=0.1,
                max_tokens=4000,
            )
            text = rsp.choices[0].message.content
            usage["input_tokens"]  = rsp.usage.prompt_tokens
            usage["output_tokens"] = rsp.usage.completion_tokens
            try:
                details = rsp.usage.prompt_tokens_details
                usage["cached_tokens"] = details.get("cached_tokens", 0)
            except AttributeError:
                usage["cached_tokens"] = 0

        else:
            raise ValueError(f"Unknown provider {provider}")

        return text, usage

In [7]:
# --- 4. Parallel Orchestration Function ---

# ────────────────────────────────────────────────────────────────────
#  Parallel across PROBLEMS, but seed Anthropic cache with the first
#  problem before launching the rest.
# ────────────────────────────────────────────────────────────────────
async def generate_GSM8K_code_parallel(
    model_dict: Dict[str, List[str]],
    indices_to_generate: List[int],
    example_indices: List[int],
    output_dir: Path = BASE_OUTPUT_DIR,
    max_parallel_problems: int | None = None,      # None = unlimited
) -> pd.DataFrame:

    # Initialize the global Gemini client
    global gemini_client
    gemini_client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

    # Create explicit caches for all specified Google models
    global gemini_model_caches
    gemini_model_caches = {} # Initialize the dictionary
    if "google" in model_dict:
        for model_name in model_dict["google"]:
            print(f"Attempting to create cache for Google model: {model_name}...")
            try:
                cache = await gemini_client.aio.caches.create(
                    model=model_name, # Use the specific model name from the list
                    config=genai.types.CreateCachedContentConfig(
                        display_name=f'GSM8K_StaticPrefixCache_{model_name.replace("/", "_")}', # Unique display name
                        system_instruction=STATIC_PREFIX,
                        ttl="86400s", # Cache for 24 hours
                    )
                )
                gemini_model_caches[model_name] = cache.name
                print(f"Successfully created cache for {model_name}: {cache.name}")
            except Exception as e:
                print(f"CRITICAL ERROR: Failed to create Gemini cache for model '{model_name}': {e}")
                gemini_model_caches[model_name] = None # Mark as failed

    else:
        print("No 'google' provider models specified in model_dict. Skipping Gemini cache creation.")


    # choose a unique run-id right at the start  ────────────────────────
    run_ts = datetime.datetime.now(datetime.timezone.utc).strftime("%Y%m%d_%H%M%S")
    output_dir.mkdir(parents=True, exist_ok=True)
    provider_sems = {
        prov: asyncio.Semaphore(limit) for prov, limit in API_CONCURRENCY_LIMITS.items()
    }
    problem_sem = (
        asyncio.Semaphore(max_parallel_problems)
        if max_parallel_problems is not None
        else None
    )

    static_prefix = STATIC_PREFIX
    rows: list[dict] = []

    # ── coroutine for a single GSM8K problem ──────────────────────────
    async def _run_one_problem(idx: int):
        async def _guard():
            nonlocal rows
            task_prompt = build_task_prompt(idx)
            p_dir = output_dir / str(idx)
            p_dir.mkdir(parents=True, exist_ok=True)

            # launch model calls (your inner gather)
            tasks = []
            for prov, models in model_dict.items():
                for m in models:
                    t = asyncio.create_task(
                        call_model_api_async(
                            provider=prov,
                            model=m,
                            static_prefix=static_prefix,
                            all_gemini_model_caches=gemini_model_caches, # Pass the dictionary of caches
                            task_prompt=task_prompt,
                            semaphore=provider_sems[prov],
                        )
                    )
                    t.meta = {"provider": prov, "model": m, "index": idx, "start": time.time()}
                    tasks.append(t)

            results = await asyncio.gather(*tasks, return_exceptions=True)
            for t, res in zip(tasks, results):
                meta = t.meta
                elapsed = time.time() - meta["start"]
                status = "Failed"
                text = ""
                usage = {"input_tokens": 0, "output_tokens": 0, "cached_tokens": 0}

                if not isinstance(res, Exception):
                    text, usage = res
                    status = "Success"
                    if text:
                        (p_dir / f"{meta['provider']}_{meta['model']}.txt").write_text(
                            text, encoding="utf-8"
                        )
                else:
                    print(f"{meta['provider']}_{meta['model']} ❌ {res}")

                rows.append(
                    {
                        "provider": meta["provider"],
                        "model": meta["model"],
                        "index": meta["index"],
                        "status": status,
                        "time_s": round(elapsed, 3),
                        "utc_completed": datetime.datetime.utcnow().isoformat(timespec="seconds"),
                        "input_tokens": usage["input_tokens"],
                        "output_tokens": usage["output_tokens"],
                        "cached_tokens": usage["cached_tokens"],
                    }
                )

        return await _guard() if problem_sem is None else await problem_sem.__aenter__() or await _guard() or await problem_sem.__aexit__(None, None, None)

    # ── 1. Seed cache with the FIRST index (blocking) ─────────────────
    first_idx, *rest = indices_to_generate
    await _run_one_problem(first_idx)

    # ── 2. Fan-out remaining problems concurrently ────────────────────
    await asyncio.gather(*[_run_one_problem(i) for i in rest])

    # Clean up: Delete all created caches
    if gemini_model_caches:
        print("\nDeleting Gemini caches...")
        delete_tasks = []
        for model_name, cache_id in gemini_model_caches.items():
            if cache_id: # Only attempt to delete if a cache ID was successfully created
                delete_tasks.append(gemini_client.aio.caches.delete(name=cache_id))

        if delete_tasks:
            await asyncio.gather(*delete_tasks, return_exceptions=True)
            print("Gemini caches deletion attempts completed.")
        else:
            print("No Gemini caches to delete.")

    # ── save CSV ─────────────────────────────────────────────────────
    df = pd.DataFrame(rows)
    csv_path = output_dir / f"generation_performance_{run_ts}.csv"
    df.to_csv(csv_path, index=False)
    print(f"✓ All done — log at {csv_path}")
    return df

# # Add any problem indices you have generated outputs for.
# problem_indices_to_test = sorted([3331, 1647, 636, 399, 4670, 5918, 1531, 7364, 5464, 1205, 3518, 6732, 3779, 4483, 6237, 1202, 2345])

In [8]:
# --- 5. Execution ---

# Define your parameters here
indices = [310, 3822, 7371] # Use the indices of your few-shot examples
indices_to_generate = list(range(100))

model_dict = {
  "anthropic": ["claude-3-5-haiku-20241022"],
  "openai": ["gpt-4.1-mini"],
  "google": ["gemini-2.5-flash-lite-preview-06-17",
             "gemini-2.5-flash"]
}

# REFACTOR: To run the async function, you must `await` it.
# This will execute the entire parallel generation process.
# The result (a pandas DataFrame with performance logs) will be stored in `perf_df`.

perf_df = await generate_GSM8K_code_parallel(
    model_dict=model_dict,
    indices_to_generate=indices_to_generate,
    example_indices=indices
)


# print("\nFinal Performance Summary:")
# print(perf_df)

Attempting to create cache for Google model: gemini-2.5-flash-lite-preview-06-17...
Successfully created cache for gemini-2.5-flash-lite-preview-06-17: cachedContents/dob8081ne1rfa8tlckjq3z4157bmts276c3h8ots
Attempting to create cache for Google model: gemini-2.5-flash...
Successfully created cache for gemini-2.5-flash: cachedContents/glikqsospwhoivvp9cgqkk07f1qqv7lxq5l944ao
anthropic_claude-3-5-haiku-20241022 ❌ Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': 'This request would exceed the rate limit for your organization (124dd641-0a24-4883-9fd3-6bed401adf9f) of 50,000 input tokens per minute. For details, refer to: https://docs.anthropic.com/en/api/rate-limits. You can see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase.'}}
anthropic_claude-3-5-haiku-202

In [9]:
perf_df

,provider,model,index,status,time_s,utc_completed,input_tokens,output_tokens,cached_tokens
0,anthropic,claude-3-5-haiku-20241022,0,Success,7.275,2025-06-25T06:43:00,135,175,0
1,openai,gpt-4.1-mini,0,Success,7.276,2025-06-25T06:43:00,3159,120,0
2,google,gemini-2.5-flash-lite-preview-06-17,0,Success,7.276,2025-06-25T06:43:00,3586,137,3461
3,google,gemini-2.5-flash,0,Success,7.276,2025-06-25T06:43:00,3586,125,3461
4,anthropic,claude-3-5-haiku-20241022,1,Success,3.698,2025-06-25T06:43:04,139,142,3416
...,...,...,...,...,...,...,...,...,...
395,google,gemini-2.5-flash,98,Success,350.538,2025-06-25T06:48:51,3704,209,3461
396,anthropic,claude-3-5-haiku-20241022,99,Success,361.022,2025-06-25T06:49:02,309,292,3416
397,openai,gpt-4.1-mini,99,Success,361.022,2025-06-25T06:49:02,3313,268,0
398,google,gemini-2.5-flash-lite-preview-06-17,99,Success,361.022,2025-06-25T06:49:02,3754,370,3461


In [10]:
import os
import re
import shutil

# Matches  ```python … ```  or  ```py … ```  (any EOL style)
_CODE_BLOCK_RE = re.compile(
    r"```(?:python|py)\s*\r?\n(.*?)\r?\n```",
    re.DOTALL | re.IGNORECASE,
)

def process_and_clean_outputs(
    source_dir: Path | str = BASE_OUTPUT_DIR,
    dest_dir:   Path | str = PROJECT_ROOT / "data" / "code_gen_outputs_cleaned",
):
    """
    Traverses `source_dir`, cleans raw .txt model outputs, and writes them as
    .py files to `dest_dir`, preserving the relative folder structure.

    Steps performed
    ---------------
    1.  Walk through every sub-directory of `source_dir`.
    2.  Detect all files that end with `.txt`.
    3.  Build the matching sub-directory path in `dest_dir`.
    4.  Read the .txt and extract code inside a markdown fence:
            ```python
            <code>
            ```
        – If no fence is found, the whole file is used as is.
    5.  Write the cleaned code to `<same_name>.py` in the destination tree.
    6.  Leave the original raw files untouched.
    """
    # Resolve to absolute paths
    source_dir = Path(source_dir).expanduser().resolve()
    dest_dir   = Path(dest_dir).expanduser().resolve()

    print(f"Starting processing from: {source_dir}")
    print(f"Cleaned files will be saved in: {dest_dir}\n")

    if not source_dir.is_dir():
        print(f"Error: Source directory '{source_dir}' not found.")
        return

    files_processed = 0

    # 1. Walk through every *.txt under source_dir
    for txt_path in source_dir.rglob("*.txt"):

        try:
            # 2. Derive the destination sub-folder (Step 3)
            rel_path   = txt_path.parent.relative_to(source_dir)
            dest_subdir = dest_dir / rel_path
            dest_subdir.mkdir(parents=True, exist_ok=True)

            # 3. Read raw content (Step 4)
            raw_content = txt_path.read_text(encoding="utf-8")

            match = _CODE_BLOCK_RE.search(raw_content.lstrip())
            if match:
                cleaned_code = match.group(1).strip()
                print(f"✓ extracted code block → {txt_path}")
            else:
                cleaned_code = raw_content.strip()
                print(f"⚠ no code block; using full file → {txt_path}")

            # 4. Write to .py in destination (Step 5)
            dest_file = dest_subdir / f"{txt_path.stem}.py"
            dest_file.write_text(cleaned_code, encoding="utf-8")
            files_processed += 1

        except Exception as e:
            print(f"❌ error processing {txt_path}: {e}")

    # 5. Summary (Step 6)
    print(f"\nProcessing complete — {files_processed} files written.")


In [11]:
process_and_clean_outputs()

Starting processing from: /content/content/data/code_gen_outputs_raw
Cleaned files will be saved in: /content/content/data/code_gen_outputs_cleaned

✓ extracted code block → /content/content/data/code_gen_outputs_raw/18/anthropic_claude-3-5-haiku-20241022.txt
✓ extracted code block → /content/content/data/code_gen_outputs_raw/18/google_gemini-2.5-flash.txt
✓ extracted code block → /content/content/data/code_gen_outputs_raw/18/google_gemini-2.5-flash-lite-preview-06-17.txt
✓ extracted code block → /content/content/data/code_gen_outputs_raw/18/openai_gpt-4.1-mini.txt
✓ extracted code block → /content/content/data/code_gen_outputs_raw/58/anthropic_claude-3-5-haiku-20241022.txt
✓ extracted code block → /content/content/data/code_gen_outputs_raw/58/google_gemini-2.5-flash.txt
✓ extracted code block → /content/content/data/code_gen_outputs_raw/58/google_gemini-2.5-flash-lite-preview-06-17.txt
✓ extracted code block → /content/content/data/code_gen_outputs_raw/58/openai_gpt-4.1-mini.txt
✓ ext